# The Parallel RETIS Game

Think you can find a better strategy for doing a parallel replica exchange TIS? Here's a simple game to test your ideas.



## Moves

So far, we only have two kinds of moves: shooting and replica exchange. Other moves, such as the minus move, might get added later to make this more realistic. To simplify both, we actually label the moves according to the **replicas** they apply to, not the ensembles. This means we don't actually have to track the replica exchange process. (Adding the minus move might require changing that).

Each move is associated with a cost function. We assume that replica exchange moves are fast, and so they have a cost of 1. Shooting moves draw their cost from a distribution. (Right now, the distribution is constant cost of 10. I plan to [SOON] make the distribution appear a bit like the path length distributions from van Erp, Moroni, and Bolhuis, JCP **118** 7762 (2003).)

## Strategies

This is where the player can (and should) design his or her own ideas. The main method a `Strategy` implements is `update(legal_moves, status, time)`, which takes as input the list of legal next moves (moves that aren't dependent on anything else happening first) and a `status` list, which shows what is happening in each node. If `status[node_number]` is `None`, then that node is free to take a new move.

The `update` function returns a list of `MoveReport`s, where `MoveReport` is a `namedtuple` with elements `move` (a move from the `legal_moves` list) and `node` (an integer for the node number). Each `MoveReport` will assign that move to the specified node. If no moves are to be assigned to a node at this time (e.g., if no nodes are free), the `update` function should return an empty list.

## Move Schemes

Of course, the nature of the move scheme can also have an effect. Fewer replica exchange choices (for example, only nearest-neighbors) mean that it is less likely that slow moves will be waiting on fast ones.

The current implementation requires that all moves be made with equal selection probability. In the future, I'll add the ability to have a different selection probability for each move. Note, however, that I won't build the whole move tree that we do in OPS: this is just the `choice_probability` defined in OPS `MoveScheme`s.

## Game

The `Game` object is also what plays the game. Internally, the game is played like this:

1. A set of `n_steps` moves is generated. This would be the order of all trial moves.
2. At each time, the strategy's `update` function is called. This determines what moves to play next.
3. Any moves to be played are assigned to a node, and removed from the list of moves to play. This continues until all moves are completed.


## Goal

There are two competing goals here: first, you want to have the smallest wall time. Second, you want to have the highest efficiency.

Getting a better wall time is trivial if you add more nodes, but that is likely to cost you in efficiency. Getting better efficiency is trivial from using fewer nodes, but that will cost you in wall time.

## Other to-dos

The stuff above mentions a few ways I intend to improve this simulations. Aside from those, the main thing that I need to do is to prevent (accidental) cheating. In order to satisfy detailed balance, we need to be sure that the effective move order does not change. That is to say, consider a first replica exchanges between replicas 1 and 2: Another exchange between replicas 3 and 4 can be done either before or after the first, but the order must be preserved if the next exchange is between 2 and 3.

The basic principle is that for any given replica, the order of moves should be the same as if it was done in serial.

# Let's play!

Here's a quick example of how to play the parallel RETIS game, using `DefaultStrategy`. The `DefaultStrategy` is to always do the next move as soon as possible. 

In [1]:
import parallel_RETIS_game as game

Now we set up the move scheme for our game. It's only fair to compare two strategies with the same move scheme. However, different move schemes shou

In [2]:
cost_shoot0 = game.ShootingCost(0,0,0)
cost_shoot1 = game.ShootingCost(1,1,1)
cost_shoot2 = game.ShootingCost(2,2,2)
shooting0 = game.ShootingMove(cost_function=cost_shoot0,
                              depends_on=[0])
shooting1 = game.ShootingMove(cost_function=cost_shoot1,
                              depends_on=[1])
shooting2 = game.ShootingMove(cost_function=cost_shoot2,
                              depends_on=[2])
shooting3 = game.ShootingMove(cost_function=cost_shoot0,
                              depends_on=[0])
shooting4 = game.ShootingMove(cost_function=cost_shoot1,
                              depends_on=[1])
shooting5 = game.ShootingMove(cost_function=cost_shoot2,
                              depends_on=[2])

repex01 = game.ReplicaExchangeMove(0, 1)
repex12 = game.ReplicaExchangeMove(1, 2)
repex34 = game.ReplicaExchangeMove(3, 4)
repex45 = game.ReplicaExchangeMove(4, 5)

moves = [shooting0, shooting1, shooting2, shooting3, shooting4, shooting5, repex01, repex12, repex34, repex45]

In [3]:
# this is where you should create your own strategy
strategy = game.DefaultStrategy()

In [4]:
mygame = game.Game(moves, strategy, n_nodes=4)
mygame.play(500)

In [5]:
mygame.analysis(mygame.history)

Wall time: 1321
CPU time: 5284
Wasted cycles: 1895
Efficiency: 0.641370174111
